In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
RAW_FILE_PATH = "../raw_data_v1/"
CITIES = [x.split('/')[-2] for x in glob.glob(RAW_FILE_PATH + "*/")]
POLLUTANTS = ['PM2.5', 'NOx', 'SO2', 'O3']
MET_PARAMETERS = ['Temp', 'RH', 'WS']

In [3]:
india_pol_data = pd.read_csv("../../2015-2020-pm25/station_day.csv")
india_pol_data['Date'] = pd.to_datetime(india_pol_data['Date'], format="%Y-%m-%d")

stations_data = pd.read_csv('../../2015-2020-pm25/stations.csv')

In [4]:
def get_xls_file_df(xls_file, file_type):
    xls_file_df = xls_file[16:]
    parameters = xls_file.iloc[15].values[2:6]
        
    try:
        if np.isnan(parameters[-1]):
            parameters = parameters[:-1]
    except:
        pass
    
    for idx, parameter in enumerate(parameters):
        if file_type == "met" and parameter == 'AT':
            parameters[idx] = 'Temp'
        elif file_type == "pol" and parameter == 'Ozone':
            parameters[idx] = 'O3'
    
    parameters = list(parameters)
     
    xls_file_df.columns = ['Date', '_1'] + parameters + ['_x' for x in range(len(xls_file_df.columns) - 2 - len(parameters))]
    
    xls_file_df = xls_file_df[['Date'] + parameters]
    xls_file_df['Date'] = pd.to_datetime(xls_file_df['Date'], format="%d-%m-%Y %H:%M")
    xls_file_df = xls_file_df.set_index('Date')
    
    return xls_file_df, parameters

In [5]:
for city_idx in range(len(CITIES)):
    
    print(50*"-")
    print(f"City : {CITIES[city_idx]}")
        
    # Extracting the raw data files for all stations belonging to each city
    raw_data_files = glob.glob(RAW_FILE_PATH + CITIES[city_idx] + "/*_pol.xlsx")
    
    for file_idx in range(len(raw_data_files)):
        
        # Reading the xls file
        xls_pol_file = pd.read_excel(raw_data_files[file_idx])
        
        try:
            xls_met_file = pd.read_excel(raw_data_files[file_idx][:-8] + "met.xlsx")
        except:
            print(f"Error: Met file not found for {raw_data_files[file_idx][:-9]}")
    
        # Extracting the station name and station ids
        station_name = xls_pol_file.iloc[11].values[0]
        
        try:
            station_id = stations_data[stations_data['StationName'] == station_name].values[0][0]
        except Exception as e:
            print(f"Error : {station_name}")
            continue
            
        # Processing the pollution dataframe
        xls_pol_file_df, pollutants = get_xls_file_df(xls_pol_file, "pol")
        
        # Processing the met dataframe
        xls_met_df, parameters = get_xls_file_df(xls_met_file, "met")

        # Extracting the old station data
        old_station_data = india_pol_data[india_pol_data['StationId'] == station_id][['Date'] + pollutants]
        old_station_data = old_station_data.set_index('Date')

        # Finding out the first date value
        first_date_value = xls_pol_file_df.index.values[0] 
        
        # Creating the final pollution dataframe
        final_df = pd.concat([old_station_data[:first_date_value-1], xls_pol_file_df])
        final_df = pd.concat([final_df, xls_met_df], axis=1)        
        
        final_df = final_df[:"2021-05-31"]  # REMOVE THIS LINE 
        
        final_df = final_df.reset_index()
        final_df['StationId'] = station_id
        final_df['StationName'] = station_name
        final_df = final_df[['StationId', 'StationName', 'Date'] + pollutants + parameters]
        final_df = final_df.replace('None', np.nan)
        final_df[pollutants] = final_df[pollutants].apply(pd.to_numeric)
        final_df[parameters] = final_df[parameters].apply(pd.to_numeric)
        
        for pollutant in POLLUTANTS:
            if pollutant not in final_df.columns:
                final_df[pollutant] = np.nan
                
        for parameter in MET_PARAMETERS:
            if parameter not in final_df.columns:
                final_df[parameter] = np.nan
                
        final_df = final_df[['StationId', 'StationName', 'Date'] + POLLUTANTS + MET_PARAMETERS]

        # Saving the final dataframe as .csv and .pkl
        file_name = CITIES[city_idx] + f'/{station_id}'            
        final_df.to_csv(file_name + '.csv')
        final_df.to_pickle(file_name + '.pkl')
        
        print(f"Saving : {station_id} - {station_name} to file")
        
    print(50*"-")

--------------------------------------------------
City : Mumbai
Error: Met file not found for ../raw_data_v1/Mumbai/chakala_andheri_east_mumbai
Error : Chakala-Andheri East, Mumbai - IITM
Saving : MH005 - Bandra, Mumbai - MPCB to file
Error: Met file not found for ../raw_data_v1/Mumbai/mazgaon_mumbai
Error : Mazgaon, Mumbai - IITM
Error: Met file not found for ../raw_data_v1/Mumbai/mulund_west_mumbai
Error : Mulund West, Mumbai - MPCB
Saving : MH006 - Borivali East, Mumbai - MPCB to file
Saving : MH011 - Sion, Mumbai - MPCB to file
Error: Met file not found for ../raw_data_v1/Mumbai/siddharth_nagar_worli_mumbai
Error : Siddharth Nagar-Worli, Mumbai - IITM
Error: Met file not found for ../raw_data_v1/Mumbai/navy_nagar_colaba_mumbai
Error : Navy Nagar-Colaba, Mumbai - IITM
Saving : MH014 - Worli, Mumbai - MPCB to file
Saving : MH009 - Kurla, Mumbai - MPCB to file
Saving : MH010 - Powai, Mumbai - MPCB to file
Error: Met file not found for ../raw_data_v1/Mumbai/deonar_mumbai
Error : Deona